# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [52]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call), data.shape, data.columns

(157.0,
 (4870, 65),
 Index([u'id', u'ad', u'education', u'ofjobs', u'yearsexp', u'honors',
        u'volunteer', u'military', u'empholes', u'occupspecific', u'occupbroad',
        u'workinschool', u'email', u'computerskills', u'specialskills',
        u'firstname', u'sex', u'race', u'h', u'l', u'call', u'city', u'kind',
        u'adid', u'fracblack', u'fracwhite', u'lmedhhinc', u'fracdropout',
        u'fraccolp', u'linc', u'col', u'expminreq', u'schoolreq', u'eoe',
        u'parent_sales', u'parent_emp', u'branch_sales', u'branch_emp', u'fed',
        u'fracblack_empzip', u'fracwhite_empzip', u'lmedhhinc_empzip',
        u'fracdropout_empzip', u'fraccolp_empzip', u'linc_empzip', u'manager',
        u'supervisor', u'secretary', u'offsupport', u'salesrep', u'retailsales',
        u'req', u'expreq', u'comreq', u'educreq', u'compreq', u'orgreq',
        u'manuf', u'transcom', u'bankreal', u'trade', u'busservice',
        u'othservice', u'missind', u'ownership'],
       dtype='object'))

In [51]:
data[['race','call']].head()

,race,call
0,w,0.0
1,w,0.0
2,b,0.0
3,w,0.0
4,b,0.0


## https://campus.datacamp.com/courses/statistical-thinking-in-python-part-2/introduction-to-hypothesis-testing?ex=4

   1. What test is appropriate for this problem? Does CLT apply?

The CLT does not apply here as the sample size is not large enough, and the observations may not be independent of each other.  For example, assume company A has 2 openings for a certain position and more than 2 people apply.  Then the probability that someone gets this position is dependent on whether or not the candidates before them got the job or not.

We will use the permutation method to assume that there is no bias in the 'call' column based on race.

   2. What are the null and alternate hypotheses?

The null hypothesis is that race has no impact on whether or not a candidate received a call.  This can be formalized as checking the difference of calls received by race. The alternate hypothesis would be that there exists a bias to candidates getting calls based on race

   3. Compute margin of error, confidence interval, and p-value.


In [53]:
orig_num_calls_w = data[(data['race']=='w') & (data['call']==1.0)].shape[0]
orig_num_calls_b = data[(data['race']=='b') & (data['call']==1.0)].shape[0]

orig_diff_calls = orig_num_calls_w - orig_num_calls_b
orig_diff_calls

78

In [47]:
#Use permutation method to permute the races within the dataframe 
num_trials = 1000
list_diff_calls = np.empty(num_trials)

for index in range(num_trials):
    #permute the race list
    race_list = data['race'].values
    race_list = np.random.permutation(race_list)

    #put it back into the dataframe
    data['race'] = race_list
    
    num_calls_w = data[(data['race']=='w') & (data['call']==1.0)].shape[0]
    num_calls_b = data[(data['race']=='b') & (data['call']==1.0)].shape[0]
    list_diff_calls[index] = num_calls_w - num_calls_b

In [58]:
#confidence interval
conf_interval = np.percentile(list_diff_calls, [2.5, 97.5])
p_val = np.sum(list_diff_calls >= orig_diff_calls)/1000
print '95% confidence interval for difference between the two races:', conf_interval
print 'p_val of the observed difference:', p_val

 95% confidence interval for difference between the two races: [-36.  38.]
p_val of the observed difference: 0


   4. Write a story describing the statistical significance in the context or the original problem.
We see from a 1000 samples that the observed difference, which is 78, is highly unlikely to happen.  Our 95% confidence interval shows that anywhere between -36 and 38 are plausible numbers.

   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
   
Our analysis shows that there is a bias when it comes to callbacks with respect to race, but this does not show that this is the most important factor in callback success.  One way to check which factors are most important to callback sucess, one can utilize a pearlson correlation test to check the correlation between each feature and the callback column.  